In [6]:
import pcshrink
import numpy as np

In [3]:
%%time
data = pcshrink.UnpackedAncestryMap("data/pca")

In [12]:
mu = np.nanmean(data.Y, axis=1)
std = np.nanstd(data.Y, axis=1)

/project/jnovembre/jhmarcus/src/miniconda3/envs/pcshrink_env/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1436: RuntimeWarning: invalid value encountered in sqrt
  std = np.sqrt(var, out=var)


In [16]:
np.nanmean((data.Y.T - mu / std).T, axis=1)

/project/jnovembre/jhmarcus/src/miniconda3/envs/pcshrink_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
/project/jnovembre/jhmarcus/src/miniconda3/envs/pcshrink_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.
/project/jnovembre/jhmarcus/src/miniconda3/envs/pcshrink_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


array([-1.4933659 , -0.2722489 , -3.4169471 , ..., -0.09382021,
               nan, -0.04309416], dtype=float32)